# Capstone Project - The Battle of the Neighborhoods 
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)



## Introduction: Business Problem <a name="introduction"></a>

### Prospects of a restaurant for lunch to the office area in London, United Kingdom

London, one of the most populous metroplitan area in the world. It is ranked first place in the global economic power index. Therefore, I beleive London is one of the best places for business entrepreneurs. 

This project aims to find the best location for lunch restaurant targeting the large pool of office workers in London. 



## Data <a name="data"></a>

1. Get London Borough data from the internet, process the information to make necessary lists.
2. Getting Coordinates of major borough using Geopy
3. Use of Foursquare location data to locate restaurants in the region


## Methodology <a name="methodology"></a>

1. Scrapping for London Borough dataset from LondonDataStore https://data.london.gov.uk/
2. Using Foursquare Location Data
3. Folium Library for data Visualization
4. Use K-Means for clustering

In [1]:
import sys
import requests
import json

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors


import io
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

from sklearn.cluster import KMeans

In [13]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_6b56ec1513e34e5491d0e6e69e483210 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='pZEi3ZrlVIWaFTgajwL4RsO3Y-GA5iaakQDQzLlhCV1N',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_6b56ec1513e34e5491d0e6e69e483210.get_object(Bucket='capstoneprojectthebattleofneighbo-donotdelete-pr-qcgy8t4qn3wxn4',Key='land-area-population-density-london.xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df= pd.read_excel(body)
df.head()


,Codes,Names,Borough,Population,Square Kilometres,Population per square kilometre
0,E09000001,City of London,City of London,7681,3.151,2437.638845
1,E05000026,Abbey,Barking and Dagenham,15297,1.279,11960.125098
2,E05000027,Alibon,Barking and Dagenham,11266,1.361,8277.736958
3,E05000028,Becontree,Barking and Dagenham,15088,1.284,11750.778816
4,E05000029,Chadwell Heath,Barking and Dagenham,11210,3.380,3316.568047


In [14]:
df.rename(columns={'Names':'Wards'}, inplace=True)

In [15]:
df.index = np.arange(1, len(df) + 1)
df.drop('Codes',axis=1, inplace=True)
df

,Wards,Borough,Population,Square Kilometres,Population per square kilometre
1,City of London,City of London,7681,3.151,2437.638845
2,Abbey,Barking and Dagenham,15297,1.279,11960.125098
3,Alibon,Barking and Dagenham,11266,1.361,8277.736958
4,Becontree,Barking and Dagenham,15088,1.284,11750.778816
5,Chadwell Heath,Barking and Dagenham,11210,3.380,3316.568047
6,Eastbrook,Barking and Dagenham,11100,3.454,3213.665316
7,Eastbury,Barking and Dagenham,12336,1.445,8537.024221
8,Gascoigne,Barking and Dagenham,14685,1.134,12949.735450
9,Goresbrook,Barking and Dagenham,12487,1.280,9755.468750
10,Heath,Barking and Dagenham,11995,2.005,5982.543641


In [18]:
# remove the last row , the total
df.drop(df.tail(1).index,inplace=True)
df

,Wards,Borough,Population,Square Kilometres,Population per square kilometre,Major_Bor_Coord
1,City of London,City of London,7681,3.151,2437.638845,"(51.5156177, -0.0919983)"
2,Abbey,Barking and Dagenham,15297,1.279,11960.125098,"(51.5541171, 0.150504342619943)"
3,Alibon,Barking and Dagenham,11266,1.361,8277.736958,"(51.5541171, 0.150504342619943)"
4,Becontree,Barking and Dagenham,15088,1.284,11750.778816,"(51.5541171, 0.150504342619943)"
5,Chadwell Heath,Barking and Dagenham,11210,3.380,3316.568047,"(51.5541171, 0.150504342619943)"
6,Eastbrook,Barking and Dagenham,11100,3.454,3213.665316,"(51.5541171, 0.150504342619943)"
7,Eastbury,Barking and Dagenham,12336,1.445,8537.024221,"(51.5541171, 0.150504342619943)"
8,Gascoigne,Barking and Dagenham,14685,1.134,12949.735450,"(51.5541171, 0.150504342619943)"
9,Goresbrook,Barking and Dagenham,12487,1.280,9755.468750,"(51.5541171, 0.150504342619943)"
10,Heath,Barking and Dagenham,11995,2.005,5982.543641,"(51.5541171, 0.150504342619943)"


In [5]:
!conda install -c conda-forge geopy --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [21]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()
df['Major_Bor_Coord']= df['Borough'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

df

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


GeocoderQuotaExceeded: HTTP Error 429: Too Many Requests

In [19]:
df[['Latitude', 'Longitude']] = df['Major_Bor_Coord'].apply(pd.Series)
df

,Wards,Borough,Population,Square Kilometres,Population per square kilometre,Major_Bor_Coord,Latitude,Longitude
1,City of London,City of London,7681,3.151,2437.638845,"(51.5156177, -0.0919983)",51.515618,-0.091998
2,Abbey,Barking and Dagenham,15297,1.279,11960.125098,"(51.5541171, 0.150504342619943)",51.554117,0.150504
3,Alibon,Barking and Dagenham,11266,1.361,8277.736958,"(51.5541171, 0.150504342619943)",51.554117,0.150504
4,Becontree,Barking and Dagenham,15088,1.284,11750.778816,"(51.5541171, 0.150504342619943)",51.554117,0.150504
5,Chadwell Heath,Barking and Dagenham,11210,3.380,3316.568047,"(51.5541171, 0.150504342619943)",51.554117,0.150504
6,Eastbrook,Barking and Dagenham,11100,3.454,3213.665316,"(51.5541171, 0.150504342619943)",51.554117,0.150504
7,Eastbury,Barking and Dagenham,12336,1.445,8537.024221,"(51.5541171, 0.150504342619943)",51.554117,0.150504
8,Gascoigne,Barking and Dagenham,14685,1.134,12949.735450,"(51.5541171, 0.150504342619943)",51.554117,0.150504
9,Goresbrook,Barking and Dagenham,12487,1.280,9755.468750,"(51.5541171, 0.150504342619943)",51.554117,0.150504
10,Heath,Barking and Dagenham,11995,2.005,5982.543641,"(51.5541171, 0.150504342619943)",51.554117,0.150504


In [ ]:
df.drop(['Major_Bor_Coord'], axis=1, inplace=True)
df

Since we want to focus only on lunch restaurant for the office workers, we need to find out few of the best business area in London. From the article [Invest in London's boroughs](https://www.business.london/invest/insights/boroughs), I figure out the best ten boroughs are 
1. Barking & Dagenham
2. Brent
3. Bromley
4. City of London
5. Croydon
6. Enfield
7. Greenwich
8. Harrow
9. Hounslow
10. Kingston

Next we will use [Foursquare](https://developer.foursquare.com/) data to obtain information on restaurants. We can then start to do analysis with our battle of neighborhoods for opening a decend restaurant in London.

In [22]:

body = client_6b56ec1513e34e5491d0e6e69e483210.get_object(Bucket='capstoneprojectthebattleofneighbo-donotdelete-pr-qcgy8t4qn3wxn4',Key='business-floorspace-rateable-value-borough.xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_2 = pd.read_excel(body)
df_2.head()


,Unnamed: 0,"RV per metre2, £/m2",Office
0,Area Code,Area,NaN
1,E09000001,City of London,301.0
2,E09000002,Barking and Dagenham,105.0
3,E09000003,Barnet,118.0
4,E09000004,Bexley,83.0


In [25]:
# get the rental value per metre for the office 
df_2.drop(df_2.head(1).index,inplace=True)
df_2

,Unnamed: 0,"RV per metre2, £/m2",Office
1,E09000001,City of London,301.0
2,E09000002,Barking and Dagenham,105.0
3,E09000003,Barnet,118.0
4,E09000004,Bexley,83.0
5,E09000005,Brent,124.0
6,E09000006,Bromley,104.0
7,E09000007,Camden,348.0
8,E09000008,Croydon,117.0
9,E09000009,Ealing,121.0
10,E09000010,Enfield,110.0


## Analysis <a name="analysis"></a>

## Results and Discussion <a name="results"></a>

## Conclusion <a name="conclusion"></a>